# import env

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# dotenv 로드
try:
    import google.colab
    print("Colab 환경에서 실행 중입니다.")
    PATH = '/content/drive/MyDrive/data/'
    env_path = PATH + "env/.env"
except ImportError:
    print("로컬 환경에서 실행 중입니다.")
    import platform
    os_name = platform.system()
    if os_name == "Windows":
        print("Windows 환경에서 실행 중입니다.")
        PATH = './'
        env_path = PATH + "env/.env"
    elif os_name == "Linux":
        print("Linux 환경에서 실행 중입니다. (Colab일 가능성 있음)")
    else:
        print(f"운영 체제: {os_name}")

load_dotenv(dotenv_path=env_path)

# getenv로 환경 변수 가져오기
# KEY 쓸 때, 띄워쓰기 하면 안됨...
# env에 있는 모든 워드들은 띄워쓰기하면 못 알아봄
api_key = os.getenv('MY_OWN_KEY')

if not api_key:
    raise ValueError(".env 파일에서 API 키를 로드하지 못했습니다.")


# 가져온 값을 environ에 저장
os.environ['OPENAI_API_KEY'] = api_key

client = OpenAI(
    api_key=api_key
)

로컬 환경에서 실행 중입니다.
Windows 환경에서 실행 중입니다.


# Text to image

In [3]:
text = "밤이 된 대도시의 고층 빌딩들, 네온 불빛으로 가득한 거리와 그 위를 비추는 밝은 보름달."

In [6]:
completion = client.images.generate(
    model = "dall-e-3", # 생성형 LLM model
    prompt = text, # 요청내용
    n = 1, # 생성 이미지 수
    size = "1024x1024", # 이미지 크기
    quality= "standard" # 생성될 이미지 품질
)

In [7]:
completion

ImagesResponse(created=1729145622, data=[Image(b64_json=None, revised_prompt='High-rise buildings in a bustling metropolis at night, streets filled with neon lights, and a bright full moon shining above.', url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-NBOwV5AVdobuycoUYOoE7VNy/user-N6rfrvplaX8hpxMC6NWPy8mI/img-agJli9ZIh0N7y132WBSlDkcS.png?st=2024-10-17T05%3A13%3A42Z&se=2024-10-17T07%3A13%3A42Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-10-16T23%3A21%3A22Z&ske=2024-10-17T23%3A21%3A22Z&sks=b&skv=2024-08-04&sig=6rp6Wp84hYexsVvnNwspEif/SKgO%2BwOly3KKKqWiWN8%3D')])

In [10]:
image_url = completion.data[0].url

In [11]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-NBOwV5AVdobuycoUYOoE7VNy/user-N6rfrvplaX8hpxMC6NWPy8mI/img-agJli9ZIh0N7y132WBSlDkcS.png?st=2024-10-17T05%3A13%3A42Z&se=2024-10-17T07%3A13%3A42Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-10-16T23%3A21%3A22Z&ske=2024-10-17T23%3A21%3A22Z&sks=b&skv=2024-08-04&sig=6rp6Wp84hYexsVvnNwspEif/SKgO%2BwOly3KKKqWiWN8%3D'

# Image to text

In [15]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    max_tokens= 50,
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "이미지를 글 50자 내로 서술"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ]
        }
    ]
)

In [18]:
response.choices[0].message.content

'화려한 도시 야경이 붉은 빛으로 물들고, 거대한 보름달이 하늘에 떠 있다. 고층 빌딩들이 반짝이며 활기찬 분위기를 자아낸다.'